# **GIẢ LẬP KIẾN TRÚC RANC VỚI TẬP MNIST**

spike generator, inspired by snntorch spikegen()

cài trực tiếp tealearning từ drive
  

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
# copy software from Google Drive to content
# !ln -s "/gdrive/My Drive/theFolder" "/content/theFolder
!cp -r /content/gdrive/MyDrive/Colab/software ./

Mounted at /content/gdrive



Tiến hành download toàn bộ code trong đường link này:
https://github.com/UA-RCL/RANC/tree/master/software
sau đó up lên colab, giải nén

Có thể sử dụng link này để download: https://minhaskamal.github.io/DownGit/#/home


In [ ]:
!unzip "software.zip"

Code này sẽ sử dụng tealayer2.0 (sử dụng tensorflow 2.x)

Sau khi giải nén, do code mặc định của họ đang sử dụng tensorflow 2.0.0b1 mà phiên bản đó giờ không còn được hỗ trợ nên cần phải chỉnh lại.

=> Truy cập "./sofware/tealayers/tealayer2.0/setup.py", chỉnh dòng 19 thành 'tensorflow-gpu==2.7.0' hoặc bất cứ phiên bản 2.x nào đang được hỗ trợ rồi lưu lại

Code của họ bị sai ở một đoạn, để sửa:

=> Đi đến "./software/tealayers/tealayer2.0/tealayer2/additivepooling.py"

Sửa dòng 73 thành "output = tf.reshape(output, [-1, int(self.num_inputs//self.num_classes), self.num_classes])" rồi lưu lại

In [11]:
%cd "./software"
!pip install "./tealayers/tealayer2.0"
!pip install "./rancutils"

[Errno 2] No such file or directory: './software'
/content/software
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./tealayers/tealayer2.0
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for tealayer2: filename=tealayer2-2.0-py3-none-any.whl size=8381 sha256=0d9d6b4000478b5ea19749a406ee60edb988f2d78d0fb36117f0e2d865e88110
  Stored in directory: /tmp/pip-ephem-wheel-cache-h8id3zei/wheels/98/3e/d2/fb82202f5627bfe136eb2273610123897f01cae8671052dba2
Successfully built tealayer2
  Attempting uninstall: tealayer2
    Found existing installation: tealayer2 2.0
    Uninstall

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./rancutils
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for rancutils: filename=rancutils-0.1-py3-none-any.whl size=9632 sha256=9cbce27014ff4f06bd537949143d943ed20968e22ae54c5ea0809d4c4db9182a
  Stored in directory: /tmp/pip-ephem-wheel-cache-o2giz29t/wheels/8d/c1/61/148622b68a98f462bf31487b2ede5ce6d3496d99ca12f2f7c1
Successfully built rancutils
  Attempting uninstall: rancutils
    Found existing installation: rancutils 0.1
    Uninstalling rancutils-0.1:
      Successfully uninstalled rancutils-0.1


# **CÀI ĐẶT CÁC THƯ VIỆN CẦN THIẾT**

In [7]:
from tealayer2 import Tea, AdditivePooling
from tensorflow.keras.layers import Flatten, Activation, Input, Lambda, concatenate
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# **KHỞI TẠO MẠNG**

Xây dựng cấu trúc của mạng

kết quả là biến "network"

In [16]:
# Greyscale images are of shape (28,28,1)
inputs = Input(shape=(28,28,1))

# Flatten the inputs so that inputs map as: flatten_input[0] -> axon[0], ..., flatten_input[255] -> axon[255]
flattened_inputs = Flatten()(inputs)

# Generate each core.
# We are taking a 16x16 square of the input image and striding it by 12. this gives us 4 cores with 0 padding encumpassing the entire image.
# Trải phẳng các ảnh input 28x28x1 thành 1 vector 784x1, quét từng đoạt 256 để đưa vào lần lượt các core, mỗi đoạn sau stride 12 phần tử so với đoạn trước
# (Trong chương 02 ở file doc là quét 4 góc bức ảnh, nó cũng tương tự như thế này)
core0 = Lambda(lambda x : x[:, :256])(flattened_inputs)
core1 = Lambda(lambda x : x[:, 176:432])(flattened_inputs)
core2 = Lambda(lambda x : x[:, 352:608])(flattened_inputs)
core3 = Lambda(lambda x : x[:, 528:])(flattened_inputs)

# Use the image distributions as corresponding inputs into our Tea Layer.
# units là số neuron được sử dụng cho tea layer này, ở đây khai báo 4 tea layer với units = 64 (tương ứng 4 core với mỗi core sử dụng 64 neuron)
core0 = Tea(units=64, name='tea_1_1')(core0)
core1 = Tea(units=64, name='tea_1_2')(core1)
core2 = Tea(units=64, name='tea_1_3')(core2)
core3 = Tea(units=64, name='tea_1_4')(core3)

# The classification is the concatenation of these 4 core's outputs.
# We'll call the classification core our 'network'
network = concatenate([core0, core1, core2, core3])

# Gộp đầu ra của 4 core trên lại thành 1 layer khác (tổng sẽ là 256 đầu vào cho tea layer này), tuy nhiên
# do chỉ có 10 class (các số từ 0 đến 9, mỗi số 1 class) => layer này chỉ sử dụng 250 neuron (do 256 không
# chia hết cho 10), trong đó cứ 25 neuron thì sẽ vote cho 1 class, số lượng neuron vote cho class nào lớn 
# nhất thì ảnh sẽ thuộc về class đó. Ví dụ trong 250 đầu ra, từ 0 đến 24 có 17 spike bắn ra, đồng thời 
# không có cụm nào bắn ra được nhiều bằng hoặc hơn 17 => Ảnh thuộc về class số "0" 
network = Tea(units=250, name='tea_2')(network)

network = AdditivePooling(10)(network)
print(network)

Tensor("additive_pooling/Sum:0", shape=(?, 10), dtype=float32)


# **TRAIN MODEL**

Lấy ảnh từ tập ảnh mnist và tạo 2 mẫu ảnh train và test

Xử lý ảnh train

Đưa mẫu ảnh train và cấu trúc mạng vào bắt đầu train. Sau đó thu được "model" đã train

### Input trực tiếp ảnh

In [4]:
# Chuẩn bị dataset để train
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(X_train.shape)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
print(y_test.shape)

11490434/11490434 [==============================] - 1s 0us/step
(60000, 28, 28)
(10000, 10)


In [ ]:
# start "softmax" function
predictions = Activation('softmax')(network)

model = Model(inputs=inputs, outputs=predictions)

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_split=0.2)

score = model.evaluate(X_test, y_test, verbose=0)

print("Test Loss: ", score[0])
print("Test Accuracy: ", score[1])

(10000, 10)
Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 2s 37us/sample - loss: 0.0993 - acc: 0.9682 - val_loss: 0.2153 - val_acc: 0.9318
Epoch 2/10
48000/48000 [==============================] - 2s 33us/sample - loss: 0.0931 - acc: 0.9693 - val_loss: 0.2044 - val_acc: 0.9350
Epoch 3/10
48000/48000 [==============================] - 2s 34us/sample - loss: 0.0920 - acc: 0.9701 - val_loss: 0.2235 - val_acc: 0.9299
Epoch 4/10
48000/48000 [==============================] - 2s 33us/sample - loss: 0.0876 - acc: 0.9716 - val_loss: 0.1859 - val_acc: 0.9395
Epoch 5/10
48000/48000 [==============================] - 2s 33us/sample - loss: 0.0863 - acc: 0.9716 - val_loss: 0.2086 - val_acc: 0.9370
Epoch 6/10
48000/48000 [==============================] - 2s 33us/sample - loss: 0.0869 - acc: 0.9724 - val_loss: 0.2013 - val_acc: 0.9405
Epoch 7/10
48000/48000 [==============================] - 2s 32us/sample - loss: 0.0842 - acc: 0.9733 - 

### Encode ảnh

In [20]:
from tealayer2 import helper_functions
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# Normalizing pixel value between 0 and 1.
X_train /= 255
X_test /= 255

X_train_encode = X_train.reshape(X_train.shape[0], (28*28))
X_test_encode = X_test.reshape(X_test.shape[0], (28*28))
# Chuẩn bị dataset để train
x_train_encoded = helper_functions.encode_dataset(X_train_encode, ticks = 4, method = 'rate encoding')
x_test_encoded = helper_functions.encode_dataset(X_test_encode, ticks = 4, method = 'rate encoding')
print(x_train_encoded.shape)
#x_test_encoded = ranc_encode_burst_rate(x_test, 1, (46, 46)).reshape(x_test.shape[0], 1, 46, 46)

#X_test /= 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


(60000, 4, 784)


In [21]:
x_train_encoded = x_train_encoded.reshape(x_train_encoded.shape[0], 28, 28, x_train_encoded.shape[1])

print(x_train_encoded.shape)

ValueError: ignored

In [22]:
x_test_encoded = x_test_encoded.reshape(x_test_encoded.shape[0], 28, 28, x_test_encoded.shape[1])

In [27]:
x_train_encoded = x_train_encoded.reshape(60000, 28, 28, 4)
x_test_encoded = x_test_encoded.reshape(10000, 28, 28, 4)
print(x_train_encoded.shape)
print(x_test_encoded.shape)

(60000, 28, 28, 4)
(10000, 28, 28, 4)


In [29]:
# start "softmax" function
predictions = Activation('softmax')(network)

model = Model(inputs=inputs, outputs=predictions)

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
x_train_encoded = x_train_encoded.reshape(60000, 4, 28, 28, 1)
x_test_encoded = x_test_encoded.reshape(10000, 4, 28, 28, 1)
model.fit(x_train_encoded, y_train, batch_size=128, epochs=10, verbose=1, validation_split=0.2)

score = model.evaluate(X_test, y_test, verbose=0)

print("Test Loss: ", score[0])
print("Test Accuracy: ", score[1])

ValueError: ignored

# **GENERATE MẠNG VÀ CÁC PACKET**

In [ ]:
# Optionally, then save the generated network out for use in the simulator and/or hardware
from rancutils.teaconversion import create_cores, create_packets, Packet
from rancutils.output_bus import OutputBus
from rancutils.serialization import save as sim_save

x_test_flat = X_test.reshape((10000, 784))
partitioned_packets = []

#test với 10000 ảnh
num_test_samples = 10
# Tạo các core bằng hàm create_cores(), sử dụng 2 layer (1 layer 4 core và 1 layer 1 core), sử dụng absolute reset mode: neuron_reset_type=0
cores_sim = create_cores(model, 2, neuron_reset_type=0) 
# Partition the packets into groups as they will be fed into each of the input cores
partitioned_packets.append(x_test_flat[:num_test_samples, :256])
partitioned_packets.append(x_test_flat[:num_test_samples, 176:432])
partitioned_packets.append(x_test_flat[:num_test_samples, 352:608])
partitioned_packets.append(x_test_flat[:num_test_samples, 528:])
# Tạo packet bằng hàm create_packets()
packets_sim = create_packets(partitioned_packets)
# Tạo output_bus bằng hàm OutputBus(coordinate, num_outputs)
output_bus_sim = OutputBus((0, 2), num_outputs=250)

# Đây chính là file đầu vào cho giả lập kiến trúc RANC bằng code C++
sim_save("mnist_config.json", cores_sim, packets_sim, output_bus_sim, indent=2)
# Lưu lại đầu ra của tensorflow predictions và correct labels để tý làm cross validation
predict = model.predict(X_test[:num_test_samples,:])
idx = []
for i in predict:
  idx.append(np.argmax(i))
test_predictions = to_categorical(idx)
np.save("mnist_tf_preds.txt", test_predictions)
np.save("mnist_correct_preds.txt", y_test[:num_test_samples,:])

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


**Lưu file mem**

In [ ]:
from rancutils.emulation import output_for_testbench, output_for_streaming

In [ ]:
output_for_streaming(cores_sim,max_xy=(4,2),output_path="/content/mnist_5_core_mem")

In [ ]:
!zip -r /content/mnist_5_core_mem.zip /content/mnist_5_core_mem

  adding: content/mnist_5_core_mem/ (stored 0%)
  adding: content/mnist_5_core_mem/tc_005.mem (deflated 99%)
  adding: content/mnist_5_core_mem/tc_001.mem (deflated 98%)
  adding: content/mnist_5_core_mem/csram_003.mem (deflated 96%)
  adding: content/mnist_5_core_mem/tc_004.mem (deflated 98%)
  adding: content/mnist_5_core_mem/csram_001.mem (deflated 96%)
  adding: content/mnist_5_core_mem/csram_000.mem (deflated 96%)
  adding: content/mnist_5_core_mem/csram_002.mem (deflated 96%)
  adding: content/mnist_5_core_mem/tc_007.mem (deflated 99%)
  adding: content/mnist_5_core_mem/tc_000.mem (deflated 98%)
  adding: content/mnist_5_core_mem/tc_003.mem (deflated 98%)
  adding: content/mnist_5_core_mem/csram_005.mem (deflated 99%)
  adding: content/mnist_5_core_mem/csram_007.mem (deflated 99%)
  adding: content/mnist_5_core_mem/csram_004.mem (deflated 87%)
  adding: content/mnist_5_core_mem/csram_006.mem (deflated 99%)
  adding: content/mnist_5_core_mem/tc_006.mem (deflated 99%)
  adding: con

In [ ]:
from google.colab import files
files.download("/content/mnist_5_core_mem.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Lưu input và output chuẩn**

In [ ]:
# lưu input và output chuẩn
output_for_testbench(packets_sim,
                         y_test[:num_test_samples,:],
                         output_path='/content/',
                         input_filename='tb_input.txt',
                         correct_filename='tb_correct.txt',
                         num_inputs_filename='tb_num_inputs.txt',
                         num_outputs_filename='tb_num_outputs.txt',
                         max_packet_xy=(512, 512),
                         num_axons=256,
                         num_ticks=16,
                         num_outputs=250)

Sau khi chạy xong đoạn code này, 3 file mới sẽ được tạo ra. Tiến hành tải file "mnist_config.json" về để làm input file cho code RANC simulator C++

# **C++**

**Compile code**

In [ ]:
cd "/content/"

/content


In [ ]:
!unzip "simulator.zip"

In [ ]:
cd "./simulator/"

/content/simulator


In [ ]:
!mkdir build

In [ ]:
cd build

/content/simulator/build


In [ ]:
!cmake ".."
!make

**Chạy code**

Muốn chạy với x ảnh thì số tick là x + 2, do trễ 2 tick từ 2 layer

In [ ]:
!/content/simulator/build/ranc_sim -i /content/software/mnist_config.json -o /content/simulator_output.txt -c /content/simulator/config.json 12

# **SO SÁNH ĐẦU RA TỪ SIMULATOR VỚI ĐẦU RA CỦA TENSORFLOW**

Tiến hành upload file output của simulator lên colab, chạy đoạn code này để so sánh

Lưu ý:

- Tên file đặt là "simulator_output.txt" và lưu trong đường dẫn hiện tại đang cd đến

- Trước khi upload, xóa 2 dòng đầu tiên trong file output đi (chi tiết trong file doc)

In [ ]:
    # TODO: Add usage example for outputting to emulation via rancutils.emulation.write_cores, etc.

    ###
    # ...
    # Start the simulator, etc, and collect results...
    # ...
    ###

    # Process the output to collect final classificaiton results and compare against Tensorflow predictions
    from rancutils.simulator import collect_classifications_from_simulator
    tf_output = np.load("/content/software/mnist_tf_preds.txt"+".npy")
    correct_output = np.load("/content/software/mnist_correct_preds.txt"+".npy")
    simulator_output = collect_classifications_from_simulator("/content/simulator_output.txt", num_classes=10)
    
    tf_output_flat = np.array([0] * tf_output.shape[0])
    for i in range(tf_output.shape[0]):
        output_i = tf_output[i, :]
        decision = np.where(output_i == max(output_i))[0]
        if len(decision) > 1:
            decision = decision[0]
        tf_output_flat[i] = decision
    tf_output = tf_output_flat
    
    correct_output_flat = np.array([0] * correct_output.shape[0])
    for i in range(correct_output.shape[0]):
        output_i = correct_output[i, :]
        decision = np.where(output_i == max(output_i))[0]
        if len(decision) > 1:
            decision = decision[0]
        correct_output_flat[i] = decision
    correct_output = correct_output_flat

    if all(tf_output == simulator_output):
        print("Tensorflow output matches simulator output exactly!")
        print(f"Testing accuracy against known class labels is {(len(np.where(tf_output == correct_output[:len(tf_output)])[0]) / len(tf_output)) * 100}%")
    else:
        print("There are differences between Tensorflow and the simulator...")
        print(f"Differences are in indices {np.where(tf_output != simulator_output)}")
        print(f"Tensorflow thought the classes were {tf_output[np.where(tf_output != simulator_output)]}")
        print(f"The simulator thought the classes were {simulator_output[np.where(tf_output != simulator_output)]}")
        print(f"The correct classes were {correct_output[np.where(tf_output != simulator_output)]}")

Tensorflow output matches simulator output exactly!
Testing accuracy against known class labels is 90.0%
